In [1]:
import SimpleITK as sitk 
import numpy as np 
from dicom_to_cnn.model.fusion.Fusion_2 import Fusion 

path_pet = r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_PT.nii'
path_mask= r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_mask_3d.nii'
path_ct = r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_CT.nii'

ct_img = sitk.ReadImage(path_ct)
pet_img = sitk.ReadImage(path_pet)
mask_img = sitk.ReadImage(path_mask)

In [2]:
print(ct_img.GetSize())
print(pet_img.GetSize())
print(mask_img.GetSize())

(512, 512, 287)
(128, 128, 287)
(128, 128, 287)


In [6]:
target_size = (128, 128, 256)
target_spacing = (4.0, 4.0, 4.0)
target_direction = (1,0,0,0,1,0,0,0,1)
objet = Fusion()
objet.set_origin_image(pet_img)
objet.set_target_volume(target_size, target_spacing, target_direction)
resampled_pet_img = objet.resample(pet_img, defaultValuePixel = -1000.0)
print(resampled_pet_img.GetSize())

(128, 128, 256)


In [18]:
from scipy.stats import truncnorm
pet_array = sitk.GetArrayFromImage(sitk.ReadImage(path_pet))
mask_array = sitk.GetArrayFromImage(sitk.ReadImage(path_mask))
print(pet_array.shape)
print(mask_array.shape)
mask_array = np.transpose(mask_array, (3,0,1,2))
print(mask_array.shape)
lower, upper = 0.33, 0.60
mu, std = 0.42, 0.06
a, b = (lower - mu) / std, (upper - mu) / std

for num_slice in range(3, mask_array.shape[0]):
    lower, upper = 0.33, 0.60
    mu, std = 0.42, 0.06
    a, b = (lower - mu) / std, (upper - mu) / std
    mask_slice = mask_array[num_slice]  # R.O.I
    roi = pet_array[mask_slice > 0]
    print(" pet :", roi[-20:])
    print('')
    n = roi / np.max(roi)
    print("pet / np.max(pet) :", n[-20:])
    print('')
    roi_trunc = truncnorm.cdf(roi / np.max(roi), a, b, loc=mu, scale=std)
    print("pet trunc :", roi_trunc[-20:])

(287, 128, 128)
(287, 128, 128, 4)
(4, 287, 128, 128)
 pet : [0.51949181 0.73331306 1.63131293 2.94115327 3.65347926 4.76085606
 3.40138746 2.56708954 2.72634439 1.53847599 1.01626821 0.9394802
 0.86688962 1.20564569 1.89352791 3.04263195 3.70458896 3.23768828
 3.37768442 0.85726026]

pet / np.max(pet) : [0.06057394 0.08550598 0.19021483 0.34294521 0.42600406 0.55512673
 0.39660957 0.29932852 0.317898   0.17938983 0.1184992  0.10954555
 0.10108132 0.14058105 0.22078969 0.35477786 0.43196356 0.37752187
 0.39384574 0.09995851]

pet trunc : [0.         0.         0.         0.03511393 0.50764723 0.98840181
 0.30211108 0.         0.         0.         0.         0.
 0.         0.         0.         0.07694786 0.54967959 0.18530498
 0.28400269 0.        ]
